<a href="https://colab.research.google.com/github/pstanisl/mlprague-2021/blob/main/01_introduction_to_bandits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MLPrague 2021

## How to Make Data-Driven Decisions: The Case for Contextual Multi-armed Bandits

### Petr Stanislav & Michal Pleva

<!-- ## Introduction

The Multi-Armed Bandit problem (MAB) is a special case of Reinforcement Learning: an agent collects rewards in an environment by taking some actions after observing some state of the environment. The main difference between general RL and MAB is that in MAB, we assume that the action taken by the agent does not influence the next state of the environment. Therefore, agents do not model state transitions, credit rewards to past actions, or "plan ahead" to get to reward-rich states.

As in other RL domains, the goal of a MAB agent is to find a policy that collects as much reward as possible. It would be a mistake, however, to always try to exploit the action that promises the highest reward, because then there is a chance that we miss out on better actions if we do not explore enough. This is the main problem to be solved in (MAB), often called the exploration-exploitation dilemma.

Bandit environments, policies, and agents for MAB can be found in subdirectories of [tf_agents/bandits](https://github.com/tensorflow/agents/tree/master/tf_agents/bandits). 
-->
## What is the magic behind RL?

Reinforcement learning (RL) is a general framework where agents learn to perform actions in an environment to maximize a reward. The two main components are the **environment**, which represents the problem to be solved (real world), and the **agent**, representing the learning algorithm.

The agent and environment continuously interact with each other. See picture below. At each time step, the agent takes an action on the environment based on its policy $\pi(a_{t}|s_{t})$, where 𝑠𝑡 is the current observation from the environment and receives a reward $r_{t+1}$ and the next observation $s_{t+1}$ from the environment. **The goal is to improve the policy so as to maximize the sum of rewards (return).**

The "magic" is hiding in the fact that the agent is learning from the received rewards. Compared to standard machine learning methods (mainly supervised learning), we do not need to know all possible outcomes.

<center>
  <img src="https://miro.medium.com/max/1400/1*7cuAqjQ97x1H_sBIeAVVZg.png" alt="rl-principle" width=600 />
</center>

For better understanding is to important define some key terms that describe the basic elements of an RL problem are:

* Environment — Physical world in which the agent operates;
* State — Current situation of the agent;
* Reward — Feedback from the environment;
* Policy — Method to map agent’s state to actions;
* Value — Future reward that an agent would receive by taking an action in a particular state;

## Multi-Armed Bandits

Multi-Armed Bandit (MAB) is a Machine Learning framework in which an agent has to select actions (arms) in order to maximize its cumulative reward in the long term. In each round, the agent receives some information about the current state (context), then it chooses an action based on this information and the experience gathered in previous rounds. At the end of each round, the agent receives the reward assiociated with the chosen action.

Perhaps the purest example is the problem that lent its name to MAB: imagine that we are faced with $k$ slot machines (one-armed bandits), and we need to figure out which one has the best payout, while not losing too much money.

<center>
  <img src="https://blogs.mathworks.com/images/loren/2016/multiarmedbandit.jpg" alt="slot machines" />
</center>

Trying each machine once and then choosing the one that paid the most would not be a good strategy: The agent could fall into choosing a machine that had a lucky outcome in the beginning but is suboptimal in general. Instead, the agent should repeatedly come back to choosing machines that do not look so good, in order to collect more information about them. This is the main challenge in Multi-Armed Bandits: the agent has to find the right mixture between exploiting prior knowledge and exploring so as to avoid overlooking the optimal actions.

More practical instances of MAB involve a piece of side information every time the learner makes a decision. We call this side information "context" or "observation".


### How it relates to Multi-Armed Bandits?

In the general RL case, the next observation $s_{t+1}$ depends on the previous state $s_{t}$ and the action $a_{t}$ taken by the policy. This last part is what separates MAB from RL: in MAB, the next state, which is the observation, does not depend on the action chosen by the agent.

This similarity allows us to solve it with the same principle used in general RL.

* An **environment** outputs observations, and responds to actions with rewards.
* A **policy** outputs an action based on an observation, and
* An **agent** repeatedly updates the policy based on previous observation-action-reward tuples.

### Standard way how we think about MAB

In traditional A/B testing methodologies, traffic is evenly split between two variations (both get 50%). Multi-armed bandits allow us to dynamically allocate traffic to variations that are performing well while allocating less and less traffic to underperforming variations. Multi-armed bandits are known to produce faster results since there’s no need to wait for a single winning variation. With the help of a relevant user data stream, multi-armed bandits can become context-based. Contextual bandit algorithms rely on an incoming stream of user context data, either historical or fresh, which can be used to make better algorithmic decisions in real-time.

![](https://www.dynamicyield.com/wp-content/uploads/2015/02/AB-vs-Bandit-Testing.png)

> Most of the description was kindly borrowed from [Tenserflow Agents overview](https://www.tensorflow.org/agents/overview).

## Example

In [2]:
from typing import Dict, List, Tuple

import matplotlib.pyplot as plt
import numpy as np

Example of the Greedy Multi-Armed Bandit

In [5]:
class BannerEnvironment(object):
  """Example of environment for banners (with Bernoulli distribution of CTR)"""
  def __init__(self, params: List[float]):
    self._params = params
    self._observe()

  def reset(self):
    return self._observe()

  def _observe(self) -> List[float]:
    self._observation = np.random.rand(1)
    return self._observation

  def step(self, action: int) -> Tuple[int, float]:
    ret = 0 if self._observe()[0] > self._params[action] else 1
    return (ret, self._observation[0])
  
  def best_action(self):
    return np.argmax(self._params)


class GreedyPolicy(object):
  """Simple greedy policy"""

  def __init__(self, values):
    self._values = values

  def action(self) -> int:
    return np.argmax(self._values)


class GreedyAgent(object):
  """Greedy Agent with optimistic initialization."""

  def __init__(self, n: int):
    self._n = n

    self.reset()

    self.policy = GreedyPolicy(self._values)

  def reset(self):
    self._counts = [0] * self._n
    self._values = [1.0] * self._n

  def train(self, experience: Dict[str, int]):
    action = experience['action']
    reward = experience['reward']

    self._counts[action] += 1

    value = self._values[action]
    n = self._counts[action]

    self._values[action] = ((n - 1) / n) * value + (1 / n) * reward

In [16]:
environment = BannerEnvironment([0.25, 0.4, 0.67])
environment.reset()

agent = GreedyAgent(3)

for _ in range(100):
  action = agent.policy.action()  
  reward, _= environment.step(action) 
  # Create trajectory nested 
  experience = {'action': action, 'reward': reward}
  # Train policy in the agent
  agent.train(experience)

print(f'Agent\'s reward estimations={agent._values} and counts={agent._counts}')

Agent's reward estimations=[0.0, 0.5, 0.6082474226804119] and counts=[1, 2, 97]
